<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TVyeca...


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [ ]:
display_markdown(user_code, raw=True)
# Output below...

Certainly! Below is a Python implementation that demonstrates how to request an API using the `requests` library. This example shows a simple GET request to a public API and handles the response.

```python
import requests

def fetch_data_from_api(url):
    try:
        # Send GET request to the specified URL
        response = requests.get(url)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Parse and return the JSON response if the request was successful
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Handle HTTP errors
    except Exception as err:
        print(f"An error occurred: {err}")  # Handle other exceptions

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a real API URL
    data = fetch_data_from_api(api_url)

    if data:
        print("Data fetched successfully:")
        print(data)
```

### Explanation:
1. **Importing Libraries**: The `requests` library is imported to make HTTP requests.
2. **Function Definition**: `fetch_data_from_api(url)` is defined, which takes the API URL as an argument.
3. **Sending GET Request**: The function uses `requests.get(url)` to send a GET request to the specified URL.
4. **Error Handling**: 
   - `response.raise_for_status()` raises an exception for HTTP error responses.
   - Two `except` blocks catch and handle `HTTPError` and other exceptions.
5. **Return JSON Response**: If successful, the function returns the parsed JSON data.
6. **Main Block**: The script includes a conditional block to run the function and print the results.

Make sure to replace `https://api.example.com/data` with a valid API endpoint to see actual results!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [ ]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

Your implementation of the API request using the `requests` library is clear, and it effectively handles both HTTP errors and general exceptions. Here are some critiques and recommendations for improvement:

### Critique

1. **Error Handling**:
   - While you handle HTTP errors and general exceptions, the current implementation does not provide the caller with enough feedback on what went wrong. Printing the error messages is useful, but the function could also return `None` or raise specific exceptions which would enable the caller to handle errors programmatically.

2. **Output Handling**:
   - The `data` variable is assumed to contain successful responses only. If an error occurs and `data` is `None`, the `if data:` check is correctly placed, but it may lead to a lack of clarity about whether the `fetch_data_from_api` function encountered an error.

3. **Code Documentation**:
   - While the function and logic are generally understandable, including docstrings in the function would improve clarity for other developers or for future maintenance. Additional detail on the expected format of `url` and the structure of the returned JSON can be helpful.

4. **Separation of Concerns**:
   - The function could be refactored to separate the fetching of data from error handling and printing. This modular approach makes your function more reusable and testable.

5. **Validation of Input**:
   - There's no validation for the `url` that is passed to the function. Although requests will handle it to an extent, it's good practice to ensure that the URL is a well-formed string before trying to make a request.

### Recommendations

1. **Refactor Error Handling**:
   ```python
   class APIError(Exception):
       pass
   
   def fetch_data_from_api(url):
       if not isinstance(url, str) or not url.startswith("http"):
           raise ValueError("Invalid URL")

       try:
           response = requests.get(url)
           response.raise_for_status()
           return response.json()
       except requests.exceptions.HTTPError as http_err:
           raise APIError(f"HTTP error occurred: {http_err}")
       except Exception as err:
           raise APIError(f"An error occurred: {err}")
   ```

2. **Docstrings**:
   Add docstrings to your functions to document their purpose, parameters, and return values.
   ```python
   def fetch_data_from_api(url):
       """Fetch JSON data from a given API URL.
       
       Args:
           url (str): The URL of the API endpoint.
           
       Returns:
           dict or None: The parsed JSON response or None if an error occurred.
       
       Raises:
           ValueError: If the URL is invalid.
           APIError: For any error that occurred during the request.
       """
   ```

3. **Higher-Level Error Handling**:
   Ensure that you handle specific exceptions to maintain clarity in the main process and decide how to handle different types of errors.

4. **Testing**:
   Consider using unit testing to verify that your function correctly handles normal conditions as well as different error conditions.

Here is a comprehensive version incorporating these suggestions:

```python
import requests

class APIError(Exception):
    pass

def fetch_data_from_api(url):
    """Fetch JSON data from a given API URL.
    
    Args:
        url (str): The URL of the API endpoint.
        
    Returns:
        dict: The parsed JSON response.
    
    Raises:
        ValueError: If the URL is invalid.
        APIError: For any error that occurred during the request.
    """
    if not isinstance(url, str) or not url.startswith("http"):
        raise ValueError("Invalid URL")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        raise APIError(f"HTTP error occurred: {http_err}")
    except Exception as err:
        raise APIError(f"An error occurred: {err}")

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Use a valid API URL
    try:
        data = fetch_data_from_api(api_url)
        print("Data fetched successfully:")
        print(data)
    except APIError as e:
        print(e)
```

By following these recommendations, your code becomes more robust, maintainable, and easier to understand for others.

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [ ]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

Thank you for the detailed critique and the valuable suggestions! Here's the revised version of the implementation that incorporates your feedback and recommendations for improvement.

### Revised Implementation

```python
import requests

class APIError(Exception):
    """Custom exception for API errors."""
    pass

def fetch_data_from_api(url):
    """Fetch JSON data from a given API URL.
    
    Args:
        url (str): The URL of the API endpoint.
        
    Returns:
        dict: The parsed JSON response.
    
    Raises:
        ValueError: If the URL is invalid.
        APIError: For any error that occurred during the request.
    """
    if not isinstance(url, str) or not url.startswith("http"):
        raise ValueError("Invalid URL: Must be a valid HTTP or HTTPS URL.")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        raise APIError(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        raise APIError(f"Request error occurred: {req_err}")
    except Exception as err:
        raise APIError(f"An unexpected error occurred: {err}")

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a valid API URL
    try:
        data = fetch_data_from_api(api_url)
        print("Data fetched successfully:")
        print(data)
    except APIError as e:
        print(e)
    except ValueError as ve:
        print(ve)
```

### Key Improvements:
1. **Custom Exception**: Created `APIError` for handling API-specific errors, enabling more granified error management.
2. **Detailed Docstring**: Added comprehensive docstrings to the `fetch_data_from_api` function for clear usage instructions and expectations.
3. **Input Validation**: Added validation for the `url` parameter to ensure it's a valid HTTP or HTTPS URL.
4. **Specific Exception Handling**: Included handling for `requests.exceptions.RequestException` separately for more detailed error feedback, distinguishing between HTTP errors and other potential request issues.
5. **Main Block Robustness**: Handled `ValueError` in the main execution block, providing feedback on incorrect URLs.

This version enhances the original implementation by making it more modular, easier to debug, and clearer for future users. Thank you for the helpful critique!

<br>

### We can of course make this a Class...
